In [15]:
import sklearn
import pandas as pd
import numpy as np

In [167]:
# Datensatz laden

# Probeklassifikation mit: "..\Daten\Volksverhetzung_Probeannotation_GermEval2018_Training_komplett\\vvh_allg.json"
#data = data.drop(axis=1,labels="id")


# "..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\HateSpeechDe_Train_HATE_VVH.json",
# "..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\HateSpeechDe_Test_HATE_VVH.json",

# Korpora\Referenzdatensatz_HateSpeech_Deutsch\HateSpeechDe_Test_HATE_Gruppe.json

#Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Test_HATE.txt


train_data = pd.read_json("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Train_HATE.json")
train_data = train_data.drop(axis=1, labels="id")

test_data = pd.read_json("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Test_HATE.json")
test_data = test_data.drop(axis=1, labels="id")

print(train_data)

                                                   text tag1  tag2
0     Wir wollen heute auf den Ticker der @ank_ffm h...  NEG  HATE
1     @Bedb_Puetz Niemand hat die Türken gebraucht s...  NEG  HATE
2     @Mesut__A @ntvde @ntv Soll ich dir alle Polize...  NEG  HATE
3           @N24 Dank der Islamisierung unserer Heimat!  NEG  HATE
4     Die meisten Schiffbrüchigen sind doch keine Fl...  NEG  HATE
...                                                 ...  ...   ...
1784  @MarleneSchfer1 @blubberette Der Kommentar war...  NEG  HATE
1785  @DrAndreaKahl1 @DrDavidBerger @focusonline @St...  NEG  HATE
1786  @RegSprecher So sieht Meinungsfreiheit in Deut...  NEG  HATE
1787  @mjungclaus GRÜNE = Umweltschänder https://t.c...  NEG  HATE
1788  @Nobby1949Z @Holger56228648 Weg mit den allen ...  NEG  HATE

[1789 rows x 3 columns]


In [97]:
# Preprocessing: 

import re
import spacy
from spacy.lang.de.stop_words import STOP_WORDS

# from gensim.utils import tokenize # Alternativ

nlp = spacy.load("de_core_news_lg", )

def preprocess(text, langmod):
    """ Vorverarbeitung eines Strings
    Output: Liste von Tokens
    Optionen:
        - Lemmatisierung
        - Zeichensetzung entfernen
        - Mindesttokenlänge
        - Stopwortentfernung
        - alles klein geschrieben
        - Ziffern entfernen
        - Links entfernen
        - alles außer Buchstabenketten entfernen
        - 
    """

    tokens = []

    # Tokenisieren
    doc = nlp(text)

    # Zeichensetzung entfernen
    tokens = [token for token in doc if token.pos_ not in {"PROPN", "NUM", "SYM", "PUNCT"}]
    
    # Mindesttokenlänge
    minlen = 2
    tokens = [token for token in tokens if len(token) > minlen]

    # Nur Alphabetszeichenketten
    tokens = [token for token in tokens if token.text.isalpha()]

    # Stopwortentfernung
    #stopwords = STOP_WORDS.union({''})
    tokens = [token for token in tokens if token.lower() not in STOP_WORDS]

    # Lemmatisierung
    tokens = [token.lemma_ for token in tokens]


    # POS-Tags: nur Verben


    # POS-Tags: nur Substantive


    # POS-Tags: nur Verben und Substantive

    # Ausgabe als Array
    # np_array = doc.toarray(['Attributes here'])

    return tokens

    

def preprocess_corpus(corpus, langmod):
    corpus_prepped = [preprocess(text, langmod) for text in corpus]
    return corpus_prepped

In [169]:
# Preprocessing & Feature Extraction

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# Features:
# 1. Zeichen-N-Gramme, char_wb: Unigramme innerhalb von Wortgrenzen (Leerzeichen)
vectorizer = CountVectorizer(analyzer='char_wb', ngram_range=(1, 4), lowercase=False)
X = vectorizer.fit_transform(train_data["text"])
X_train = X.toarray()
X_test = vectorizer.transform(test_data["text"])
X_test = X_test.toarray()


In [133]:
# 2. doc2vec (in der gensim-Implementierung)

# s. https://radimrehurek.com/gensim/auto_examples/tutorials/run_doc2vec_lee.html


import gensim
#from gensim import corpora, models, similarities, downloader
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec


def read_corpus(datei, tokens_only=False):
    with open(datei, encoding="utf-8") as f:
        for i, line in enumerate(f):
            id, tweet, anno = line.strip().split("\t")
            # TODO: ändern, so dass nicht alles lowercase ist
            tokens = gensim.utils.simple_preprocess(tweet)
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                # Tag z.B. Zeilennummer ([i]), oder: Korpus-ID
                yield gensim.models.doc2vec.TaggedDocument(tokens, [id])

train_corpus = list(read_corpus("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\HateSpeechDe_Train_HATE_Handlung.txt"))
test_corpus = list(read_corpus("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\HateSpeechDe_Test_HATE_Handlung.txt"))


# Distributed Memory
model_dm = gensim.models.doc2vec.Doc2Vec(vector_size=300, min_count=2, epochs=30)
model_dm.build_vocab(train_corpus)
model_dm.train(train_corpus, total_examples=model_dm.corpus_count, epochs=model_dm.epochs)

# Distributed Bag of Words
model_dbow = gensim.models.doc2vec.Doc2Vec(dm=0, vector_size=300, min_count=2, epochs=30)
model_dbow.build_vocab(train_corpus)
model_dbow.train(train_corpus, total_examples=model_dbow.corpus_count, epochs=model_dbow.epochs)

# zur Kombination von DBOW und DM
concat_model = ConcatenatedDoc2Vec([model_dbow, model_dm])

# Modell speichern


# Einzelnen Vektor berechnen (für einen bereits vorverarbeiteten, tokenisierten String): 
#vector = concat_model.infer_vector(gensim.utils.simple_preprocess( "Die gr\u00f6\u00dfte Terrororganisation der Welt ist die USA und ihrer Ostk\u00fcste!"))
#print(vector)

# Alle Vektoren berechnen

# von https://towardsdatascience.com/multi-class-text-classification-with-doc2vec-logistic-regression-9da9947b43f4;
# regressors? steps?
def feature_vectors(model, docs, tags):
    targets, tags_final = zip(*[(model.infer_vector(doc.words, epochs=20), tags[line]) for line, doc in enumerate(docs)])
    return targets, tags_final

# Alternativ: mit der gensim-eigenen API für sklearn
# s. https://radimrehurek.com/gensim_3.8.3/sklearn_api/d2vmodel.html
# from gensim.sklearn_api import D2VTransformer


In [40]:
print(train_corpus[4].words)

['user', 'user', 'scheiss', 'deutsche', 'politiker', 'mehr', 'gibt', 'es', 'wohl', 'nicht', 'dazu', 'zu', 'sagen']


In [171]:
# Annotation in Zahlen konvertieren

labels_vvh = ["KEINE", "VVH"]
labels_gruppe = ["KeineGruppe", "Gruppe"]
labels_handlung = ["KeineHandlung", "Handlung"]

def labeling(label):
    if label == "NEG": return 1
    else: return 0

def labeling_vvh(label):
    if label == "VVH": return 1
    elif label == "KEINE": return 0

def labeling_gruppe(label):
    if label == "Gruppe": return 1
    elif label == "KeineGruppe": return 0

def labeling_handlung(label):
    if label == "Handlung": return 1
    elif label == "KeineHandlung": return 0

tags_train = list(map(labeling, list(train_data["tag1"])))
tags_test = list(map(labeling, list(test_data["tag1"])))

In [150]:
# Train/Test Split

# durch die Dateien vorgegeben, ca 70/30
X_train, y_train  = feature_vectors(concat_model, train_corpus, tags_train)
X_test, y_test  = feature_vectors(concat_model, test_corpus, tags_test)

# from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3,random_state=109, stratify=Y)

In [172]:
X_train, X_test, y_train, y_test = X_train, X_test, tags_train, tags_test

In [173]:
# Training
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.linear_model import LogisticRegression # war wohl ganz gut (Projekt FU)
from sklearn.ensemble import RandomForestClassifier

model = GaussianNB()
model.fit(X_train,y_train)

GaussianNB()

In [174]:
# Evaluation

from sklearn import metrics

predicted = model.predict(X_test)
# np.mean(predicted==y_test) # Accuracy

print(str(metrics.classification_report(y_test, predicted, target_names=labels_handlung)))
print(str(metrics.confusion_matrix(y_test, predicted)))

#Bayes, Probeklassifikation
"""              precision    recall  f1-score   support

    VVH-Allg       0.40      0.19      0.26        21
       Keine       0.94      0.98      0.96       286

    accuracy                           0.93       307
   macro avg       0.67      0.58      0.61       307
weighted avg       0.91      0.93      0.91       307

[[  4  17]
 [  6 280]]

 precision    recall  f1-score   support

    VVH-Allg       0.00      0.00      0.00        23
       Keine       0.93      1.00      0.96       284

    accuracy                           0.93       307
   macro avg       0.46      0.50      0.48       307
weighted avg       0.86      0.93      0.89       307
"""


ValueError: Number of classes, 1, does not match size of target_names, 2. Try specifying the labels parameter

In [ ]:
# Alternative: Cross validation

from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB


cvmodel = MultinomialNB()
scores = cross_val_score(cvmodel, X=X.toarray(), y=Y, cv=5, scoring='f1_macro')

In [ ]:
print(scores)